## Pong 04

Versión completa del juego. Usamos sprites perp para darle un aspecto más retro aun.

In [1]:
version = '4.0'

import math
import pygame
import random
import time

from pygame.locals import Rect
from math import pi
import vectores
from vectores import Vector2

SIZE = WIDTH, HEIGHT = 800, 600  # Tamaño de pantalla
CENTER = (WIDTH//2, HEIGHT//2)

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego


def draw_vector(screen, color, vector, origin=vectores.zero):
    pygame.draw.line(screen, color, origin, origin+vector)
    pygame.draw.circle(screen, color, origin, 3)
    arrow_point = origin + vector
    orientation = vector.unit() * 10
    orientation.theta -= 7*pi/8
    left_wing =  arrow_point + orientation 
    orientation.theta -= pi/4
    right_wing =  arrow_point + orientation
    pygame.draw.polygon(screen, color, [arrow_point, left_wing, right_wing])
    

class Ball(pygame.sprite.Sprite):
    
    def __init__(self, x=0, y=0):
        super().__init__() 
        self.frames = [
            pygame.image.load('pong/ball_00.png'),
            pygame.image.load('pong/ball_01.png'),
            pygame.image.load('pong/ball_02.png'),
        ]
        self.num_frames = len(self.frames)
        self.counter = 0
        self.image = self.frames[self.counter]
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        self.speed = 10
        self.reset()

    def reset(self):
        self.rect.center = CENTER
        self.orientation = Vector2(1, 0)
        self.orientation.theta = random.uniform(pi/4, -pi/4)
        
    def bounce(self):
        self.orientation.theta = -self.orientation.theta

    def update(self):
        delta =  self.orientation * self.speed
        self.rect.center = Vector2(self.rect.centerx, self.rect.centery) + delta
        if self.rect.top < 0:
            self.rect.top = 0
            self.bounce()
        elif self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT
            self.bounce()
        self.counter = (self.counter + 1) % self.num_frames
        self.image = self.frames[self.counter]

class InputState:
    
    def __init__(self):
        self.left_up = False
        self.left_down = False
        self.right_up = False
        self.right_down = False
        self.exit = False
        
    def update(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.exit = True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.exit = True
                elif event.key == pygame.K_w:
                    self.left_up = True
                elif event.key == pygame.K_s:
                    self.left_down = True
                elif event.key == pygame.K_UP:
                    self.right_up = True
                elif event.key == pygame.K_DOWN:
                    self.right_down = True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_w:
                    self.left_up = False
                elif event.key == pygame.K_s:
                    self.left_down = False
                elif event.key == pygame.K_UP:
                    self.right_up = False
                elif event.key == pygame.K_DOWN:
                    self.right_down = False


class Paddle(pygame.sprite.Sprite):
    
    def __init__(self, x, y):
        super().__init__()
        self.frames = [
            pygame.image.load('pong/paddle_00.png'),
            pygame.image.load('pong/paddle_01.png'),
            pygame.image.load('pong/paddle_02.png'),
        ]
        self.num_frames = len(self.frames)
        self.counter = 0
        self.image = self.frames[self.counter]
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        
        
    def update(self):
        self.rect = self.rect.move(0, self.speed)
        self.counter = (self.counter + 1) % self.num_frames
        self.image = self.frames[self.counter]
        
    def go_up(self):
        self.speed = -10

    def go_down(self):
        self.speed = 10
        
    def stop(self):
        self.speed = 0

        
class LeftPad(Paddle):
    
    def update(self, inputs):
        if inputs.left_up:
            self.go_up()
        elif inputs.left_down:
            self.go_down()
        else:  
            self.stop()
        super().update()
        
class RightPad(Paddle):
    
    def update(self, inputs):
        if inputs.right_up:
            self.go_up()
        elif inputs.right_down:
            self.go_down()
        else:
            self.stop()        
        super().update()   

class Game:
    
    def __init__(self, screen):
        self.screen = screen
        box = self.screen.get_rect()
        self.width, self.height = box.width, box.height
        self.background = pygame.image.load('pong/background.png')
        self.font = pygame.font.Font('pong/fonts/arcade.ttf', 90)
        self.center = Vector2(self.width // 2, self.height // 2)
        self.r_paddle = RightPad(self.width - 50, self.center.y)
        self.l_paddle = LeftPad(50, self.center.y)
        self.ball = Ball(self.center.x, self.center.y)
        self.all_objects = pygame.sprite.Group(
            self.ball, 
            self.r_paddle,
            self.l_paddle
            )
        self.score = {
            'left': 0,
            'right': 0,
            }
        self.reset()

    def reset(self):
        self.counter = 3 * FPS
        self.ball.reset()
    
    def update(self, inputs):
        self.r_paddle.update(inputs)
        self.l_paddle.update(inputs)
        self.ball.update() 
        if self.ball.rect.colliderect(self.l_paddle.rect):
            self.ball.rect.left = self.l_paddle.rect.right
            self.ball.orientation = Vector2(1, 0)
            delta = self.l_paddle.rect.centery - self.ball.rect.centery
            self.ball.orientation.theta +=  (delta * pi) / 100
            
        elif self.ball.rect.colliderect(self.r_paddle.rect):
            self.ball.rect.right = self.r_paddle.rect.left
            self.ball.orientation = Vector2(-1, 0)
            #self.ball.orientation.theta = pi - self.ball.orientation.theta

        
    def left_wins(self):
        return self.ball.rect.left < 0

    def right_wins(self):
         return self.ball.rect.right > WIDTH

    def draw(self, trace_on=False):
        self.screen.blit(self.background, (0, 0))
        self.draw_score()
        if self.counter > 0:
            seconds = self.counter // FPS
            self.draw_text(seconds, self.center)
        self.all_objects.draw(screen)
        if trace_on:
            for item in all_objects.sprites():
                pygame.draw.rect(screen, RED, item.rect, 1)
        
        pygame.draw.rect(screen, RED, self.l_paddle.rect, 2)
        delta = self.l_paddle.rect.centery -  self.ball.rect.centery
        delta = min(delta, 50) if delta > 0 else max(delta, -50)
        self.draw_text(delta, (50, 550))
        left_bounce = vectores.left
        left_bounce.theta = (delta * pi) / 200
        origin = Vector2(self.l_paddle.rect.centerx, self.l_paddle.rect.centery)
        
        draw_vector(self.screen, YELLOW, left_bounce*100, origin)
        draw_vector(self.screen, RED, self.ball.orientation*70, Vector2(self.ball.rect.centerx, self.ball.rect.centery))
        
        
        pygame.display.update()

    def draw_text(self, text, position):
        img = self.font.render(str(text), True, GREEN)
        rect = img.get_rect()
        rect.center = position
        self.screen.blit(img, rect)
        
    def draw_score(self):
        self.draw_text(self.score['left'], (200, 50))
        self.draw_text(self.score['right'], (600, 50))

    
pygame.init()
try:
    pygame.display.set_caption(version)
    screen = pygame.display.set_mode(SIZE, 0, 24)
    game = Game(screen)
    clock = pygame.time.Clock()    
    input_state = InputState()
    
    while True:
        input_state.update()
        if input_state.exit:
            break
        game.update(input_state)
        if game.left_wins():
            game.score['left'] += 1
            game.reset()
        if game.right_wins():
            game.score['right'] += 1
            game.reset()
        game.draw()
        clock.tick(FPS)
        if game.score['left'] > 15 or game.score['right'] > 15:
            break
finally:
    pygame.quit()